
Relations and categories, while not being synonymous, are linked together in my mind. Many of the interesting concrete examples of categories are relations and the canonical examples of categroical axiom sets.

Examples of categories:
- Groups
- Monoids
- Sets
- Preorders
- Lattices
- Relations


A relation is a set where some pieces of each element are dstinguished as "input" and "output".
Functions are an important example of a relation where for every possible input there is exactly one output. Functions really do have an input/output character as the two ends of a function are not interchangable.
Relations enrich the possibilites given by functions and give an algebraic closure to many of the questions one might ask.
Not all functions are invertible back into functions. Like the square root, sometimes there might be mutiple values that map to the same spot.
However, a relation always has a converse. In the case the particular relation is also an inverttible function, the converse is the inverse.
Unlike functions, Relations can be partially defined on their domain.


Very informally, in some areas, partially defined functions and multi valued functions might still be called functions, but here we're going to call these things relations.





A finding a point in a composition of many relations can be thought of as a feasibility problem.



The standard optimization framework has a little more to it though. It gives you the ability to add an objective function to minimize or maximize.
These two capabilities are close to interchangable. If you set the objective function to 0, then an optimization problem becomes a feasibility problem. And you can use a sequence of feasiblity problem to find the extermal solution via binary search.


Convex analysis has a notion of an operator.

The is actually a very natural definition to me because it corresponds to the steepest dewscent approxiation of the more usual definition of a linear operator.
Steepest descent is a method by which to approximate highly peaked and/or highly oscialltory integrals. The intergal is domoniated by it's value and beahvior in the area where it is most peaked or least oscillatory, so one yuou find these areas, you can exapnd a solvable approximation located at them. This approximation is usually a guassian integral. 
This procedure is the mathematical abstraction behind the connection between stastical mechanics and themodynamics, wave optics and geoemetrical optics, and classical mechanics and quantum mechanics. Quite a powerful little guy!
One characterization of the central problem of statistical mechanics is to evaluate the integral that defines the partition function $latex Z = \int dpdx e^{-\beta H} = e^{\beta F}$. These integrals are very hard in general.
It is perhaps a little off the beaten path to consider the boundaries of a partition function as an operator, but you will find this soprt of thing in the transfer matrix method for example for solving an Ising model.


One characterization of the central problem of quantum mechanics is to evaluate the path integral $latex \int [Dx] e^{ \frac{i}{\hbar} S[x] }$. This can approximated by instead taking the extremal path of the exponents, which then becomes the classical principle of least action.







A Cookbook for Categorfying DSLs












In [3]:
using JuMP

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1278


In [6]:
using GLPK

┌ Info: Precompiling GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1278


In [7]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [ ]:
optimize!(model)

In [ ]:
@constraint(model, con, 1x + 5y <= 3)


In [11]:
struct JuMPDomain
    it
end

struct JuMPRelation
    f
    dom::JuMPDomain
    codom::JuMPDomain
end



In [13]:
using Catlab
using Catlab.Theories

In [32]:
@instance Category{JuMPDomain, JuMPRelation} begin
  dom(M::JuMPRelation) = M.dom
  codom(M::JuMPRelation) = M.codom

  id(m::JuMPDomain) = JuMPRelation(model -> begin
         x = @variable(model,[1:m.it])
         (x, x)
         end, m, m)
  compose(M::JuMPRelation, N::JuMPRelation) =
     JuMPRelation( model -> begin 
                            (x,y) = M.f(model)
                            (y1, z) = N.f(model)
                            @constraint(model,  y1 .== y)
                            (x,z)
        end
                        ,  M.dom, N.codom)
end

In [36]:
f = compose(id(JuMPDomain(1)) , id(JuMPDomain(1)))
f.f(model)

(VariableRef[noname], VariableRef[noname])

In [37]:
@instance MonoidalCategory{JuMPDomain, JuMPRelation} begin
  otimes(A::JuMPDomain, B::JuMPDomain) = JumpDomain(A.it + B.it)
  otimes(f::JuMPRelation, g::JuMPRelation) = JumpRelation(
     model -> begin
            (x,y) = f.f(model)
            (a,b) = g.f(model)
            ( vcat([x,a]), vcat([y,b])  )
        end
    , otimes(f.dom , g.dom  ) , otimes(f.codom, g.codom)
       )
   #munit() ??? model -> 
   braid(A::JuMPDomain, B::JuMPDomain) = JumpRelation(
     model -> begin  
        x = @variable(model,[1:A.it])
        y = @variable(model,[1:B.it])
        (vcat(x,y), vcat(y,x))
        end
       otimes(A,B), otimes(B,A) )
    
   


end

LoadError: Method dom(::JuMPRelation) not implemented in mod instance

In [9]:
model -> begin 
     @variable(model, x)
     (x, x)
     end

model -> begin
         @variable(model,x)
         (x, (x, x))
         end
    
# hmm. It's mutational. That's fine.
    
model = Model(GLPK.Optimizer)


    
    

#1 (generic function with 1 method)